<a href="https://colab.research.google.com/github/pkhamchuai/namechanger_twitter_youtube/blob/main/namechanger_tw_yt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This is the twitter name changer according to view count of a youtube video.** This version of code works in colab. I made another version that doesn't need to get the channel information, but it doesn't run in colab, only runs in my local computer.
## References
*   How to create a Twitter app using Python [here](https://www.digitalocean.com/community/tutorials/how-to-create-a-twitter-app-with-python)
*   Source for Twitter name changer app [here](https://github.com/raghavkhanna30/twitter-auto-name-changer)
*   Source for extracting view count of a Youtube video [here](https://www.presentslide.in/2019/09/extracting-youtube-data-api-python.html) [github](https://github.com/sainivarsha97/youTubeApi_Tutorial)


In [ ]:
import tweepy
import logging
import time
from googleapiclient.discovery import build

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
# logger.propagate = True

youTubeApiKey='your Youtube API key'  # put in your API key here
youtube=build('youtube','v3', developerKey=youTubeApiKey)
channelId='UClIsaGq7vBEW00ASqwQyzPw'  # BNK48 channel

In [ ]:
def create_api():
    consumer_key = 'REPLACE_YOUR_KEY'         # your twitter developer key
    consumer_secret = 'REPLACE_YOUR_KEY'      # your twitter developer secret
    access_token = 'REPLACE_YOUR_KEY'         # your twitter app access token
    access_token_secret = 'REPLACE_YOUR_KEY'  # your twitter app access token secret

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True,
                     wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
    except Exception as e:
        logger.error('Error creating API', exc_info=True)
        raise e
    logger.info('API created')
    return api

In [ ]:
def get_video_info():
    # get viewcount of your desired video
    contentdata=youtube.channels().list(id=channelId,part='contentDetails').execute()
    playlist_id = contentdata['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    videos = []
    next_page_token = None

    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id, 
                                        part='snippet', 
                                        maxResults=50,
                                        pageToken=next_page_token).execute()
        videos += res['items']
        next_page_token = res.get('nextPageToken')

        if next_page_token is None:
            break
    video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))
    stats = []
    for i in range(0, len(video_ids), 40):
        res = (youtube).videos().list(id=','.join(video_ids[i:i+40]),part='statistics').execute()
        stats += res['items']
    for i in range(0, len(video_ids)):
        if video_ids[i] == 'Ek8itihPQgE': # TODO: insert youtube  video ID (โดดดิด่ง)
            result = int((stats[i])['statistics']['viewCount'])
    return result


def validate_count(user):
    # update string split if you don't use this naming format for twitter profile:
    # 'your_twitter_name: {viewcount:,} views'
    current_count = user.name.replace(':', ' ').split()
    return current_count


In [ ]:
# this cell will stay running
api = create_api()

while True:
    # change to your own twitter handle (@something),
    # the one you registered as developer with
    user = api.get_user('your_handle')
    viewcount = get_video_info()
    if validate_count(user) == viewcount:
        logger.info(
            f'Viewcount stays the same, no update necessary: {validate_count(user)} -> {viewcount}')
    else:
        logger.info(
            f'Viewcount has changed, updating twitter profile: {validate_count(user)} -> {viewcount:,}')
        # Updating your twitter profile with your name including the view count
        api.update_profile(
            name=f'your_twitter_name: {viewcount:,} views')

    logger.info("Waiting to refresh..")
    time.sleep(60 * 15) # update every 15 minutes
